- group dataset에서 해당 그룹에 속하는 유저를 다 찾아내자!

In [5]:
import pandas as pd
import numpy as np
from os.path import join
import os

In [7]:
def load_rating_file_as_list(self, filename):
    ratingList = []
    with open(filename, "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split(" ")
            user, item = int(arr[0]), int(arr[1])
            ratingList.append([user, item])
            line = f.readline()
    return ratingList

In [10]:
path="data/CAMRa2011"
group_train_file = path + '/groupRatingTrain.txt'
group_test_file = path + '/groupRatingTest.txt'
train_file = path + '/userRatingTrain.txt'
test_file = path + '/userRatingTest.txt'

In [14]:
def Dataset(file_train, user=True):
    data = []
    with open(file_train, 'r') as file:
        for line in file:
            line = line.strip().split()  
            data.append([int(x) for x in line])  

    if user:
        df = pd.DataFrame(data, columns=['user_id', 'item_id', 'rating'])
    else:
        df = pd.DataFrame(data, columns=['group_id', 'item_id', 'rating'])
    return df 

In [30]:
df_group_train = Dataset(group_train_file, False)
df_group_test = Dataset(group_test_file, False)
df_user_train = Dataset(train_file)
df_user_test = Dataset(test_file)

In [16]:
df_group_train.head()

,group_id,item_id,rating
0,159,236,70
1,278,2983,70
2,68,2832,75
3,153,3895,5
4,250,6241,55


In [18]:
df_group_test.head()

,group_id,item_id,rating
0,157,2970,80
1,25,4381,70
2,244,3900,100
3,126,2834,60
4,175,7077,70


In [19]:
df_user_train.head()

,user_id,item_id,rating
0,487,6353,20
1,30,4380,95
2,581,1571,65
3,493,2196,75
4,310,464,60


In [20]:
df_user_test.head()

,user_id,item_id,rating
0,311,6197,50
1,390,968,65
2,113,4120,20
3,198,2971,80
4,242,810,60


In [31]:
merged_df = pd.merge(df_group_train, df_user_train, on='item_id') # merge group and user data
grouped_user_ids = merged_df.groupby('group_id')['user_id'].apply(list).reset_index() # group users by group_id
df_group_train = pd.merge(df_group_train, grouped_user_ids, on='group_id', how='left') # merge group_user_ids with group data
df_group_train = df_group_train.rename(columns={'user_id': 'group_user_ids'})
df_group_train['group_user_ids'] = df_group_train['group_user_ids'].apply(lambda x: list(set(x)))

df_group_train['num_users_in_group'] = df_group_train['group_user_ids'].apply(len)
df_group_train.head()

In [ ]:
merged_df = pd.merge(df_group_test, df_user_test, on='item_id') # merge group and user data
grouped_user_ids = merged_df.groupby('group_id')['user_id'].apply(list).reset_index() # group users by group_id
df_group_test = pd.merge(df_group_test, grouped_user_ids, on='group_id', how='left') # merge group_user_ids with group data
df_group_test = df_group_test.rename(columns={'user_id': 'group_user_ids'})
df_group_test['group_user_ids'] = df_group_test['group_user_ids'].apply(lambda x: list(set(x)))

df_group_test['num_users_in_group'] = df_group_test['group_user_ids'].apply(len)
df_group_test.head()

,group_id,item_id,rating,group_user_ids,num_users_in_group
0,157,2970,80,"[152, 146, 515, 303, 118, 118, 512, 15, 344, 1...",34
1,25,4381,70,"[395, 60, 377, 58, 240, 60, 252, 584, 486, 5, ...",30
2,244,3900,100,"[49, 43, 320, 49, 49, 539, 418, 539]",8
3,126,2834,60,"[318, 524, 311, 89, 556, 129, 333, 485, 391, 6...",48
4,175,7077,70,"[552, 379, 115, 567, 379, 198, 136, 30, 439, 4...",13
